In [1]:
import csv
from pyplasm import *
from larlib import *

def intersperse(sequence, value):
    """
    Intersperse a list with an element
    
    Args:
        sequence: the list to process
        value: the element to intersperse in list
    Return:
        an interspersed list
    Example:
        >>> intersperse(['a', 'b', 'c'], '-')
            
        ['a', '-', 'b', '-', c']
    """
    
    res = [value] * (2 * len(sequence) - 1)
    res[::2] = sequence
    return res


def getCenter(base, x):
    """
    Given two numbers, base and x returns a positive value y such that
    base - y = x + y. If x is greater than base returns 0.
    
    Args:
        base: a positive real number
        x: a positive real number
    Returns:
        a positive value y such that base - y = x + y if base is greater 
        than x, 0 otherwise
    """
    
    if (base > x):
        center = (base - x) / 2.0
    else:
        center = 0
        
    return center


def createFrame(beamDimensions,
                pillarDimensions,
                axesDistances,
                interstoryHeights):

    """
    Return a HPC value describing a space frame in reinforced concrete.

    Args:
        beamDimensions: a tuple containing beam section dimension (x, z)
        pillarDimensions: a tuple containing pillar section dimension (x, y)
        axesDistances: a list containing distances between pillar axes
        interstoryHeights: a list containing interstory heights
    Returns:
        a HPC value describing the space frame parameterized by the arguments

    """

    bx, bz = beamDimensions
    px, py = pillarDimensions

    # for centering beams on pillar
    centerX = getCenter(px, bx)

    floors = []

    for height in interstoryHeights:

        pillar = CUBOID([px, py, height])
        beams = [T(1)(centerX)]
        pillars = [pillar]
        beamWidth = py / 2.0 # the first beam is half py size wider than the
                             # others so we account for this using beamWidth

        for i in range(0, len(axesDistances) - 1):
            distance = axesDistances[i]
            beamWidth += distance + py
            beams += [CUBOID([bx, beamWidth, bz]), T(2)(beamWidth)]
            pillars += [T(2)(distance + py), pillar]
            beamWidth = 0

        distance = axesDistances[-1]

        # the last beam has the same width as the first. We add beamWidth
        # to account for the case where there are two pillars.
        beams.append(CUBOID([bx, distance + py + (py / 2.0) + beamWidth, bz]))

        # last pillar
        pillars += [T(2)(distance + py), pillar]

        floors += [STRUCT(pillars), T(3)(height), STRUCT(beams), T(3)(bz)]

    return STRUCT(floors)


def generateBeams(structParams, beamX):
    """
    Returns a HPC value describing the beams that connect two space frames.

    Args:
        structParams: a list containing struct parameters (the same parameters
                      passed to the createFrame function)
        beamX: a beam dimension
    Returns:
        a HPC value describing the beams connecting two space frames

    """

    (by, bz), (px, py), distances, heights = structParams

    # get centering value
    centerY = getCenter(py, by)

    negate = lambda x: -x
    adjust = lambda x: negate(x + (2 * centerY))

    beams_y = [-centerY, by] + intersperse(map(adjust, distances), by) + [by]
    beams_z = intersperse(map(negate, heights), bz) + [bz]

    # generate beams
    res = PROD(
        [PROD([QUOTE([beamX + px]), QUOTE(beams_y)]),
         QUOTE(beams_z)])

    return res


def ggpl_bone_structure(file_name):
    """
    Create a 3D HPC value representing the bone structure of a reinforced
    concrete building
    
    Args:
        file_name: path of a csv file containing parameters to build the
        structure
    Returns:
        a HPC value representing the bone structure of a reinforced concrete
        building
    """
    state = 0
    res = []
    params = None

    # should use a less dangerous parsing method than eval
    parseVect = lambda line: eval(line)
    parseList = lambda line: map(parseVect, line)

    with open(file_name) as f:
        for row in csv.reader(f):          
            # FSM used for parsing csv. 0: odd lines, 1: even lines    
            if state == 0:
                position = parseVect(*row)
                beamX = position[0]

                if params:
                    beams = generateBeams(params, beamX)
                    res.append(beams)

                res.append(T([1,2,3])(position))
                state = 1
                
            else:
                params = parseList(row)
                res.append(createFrame(*params))
                state = 0

    return STRUCT(res)

Evaluating fenvs.py..
...fenvs.py imported in 0.010134 seconds


/usr/local/lib/python2.7/dist-packages/larlib/larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


In [2]:
VIEW(ggpl_bone_structure('frame_data_438984.csv'))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x7f3150dc2300> >